In [ ]:
from google.colab import drive
drive.flush_and_unmount()

import shutil
import os

# Path to the content directory
content_dir = '/content/'

# List the contents inside the content directory
for filename in os.listdir(content_dir):
    file_path = os.path.join(content_dir, filename)
    try:
        if os.path.isfile(file_path) or os.path.islink(file_path):
            os.unlink(file_path)  # Remove files or symlinks
        elif os.path.isdir(file_path):
            shutil.rmtree(file_path)  # Remove directories
    except Exception as e:
        print(f'Failed to delete {file_path}. Reason: {e}')

print(f"All contents inside '{content_dir}' have been deleted.")

Drive not mounted, so nothing to flush and unmount.
All contents inside '/content/' have been deleted.


In [ ]:
!nvidia-smi

Sat Apr  5 21:45:02 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  NVIDIA A100-SXM4-40GB          Off |   00000000:00:04.0 Off |                    0 |
| N/A   31C    P0             43W /  400W |       0MiB /  40960MiB |      0%      Default |
|                                         |                        |             Disabled |
+-----------------------------------------+-----

In [ ]:
%pip install ultralytics supervision roboflow
import ultralytics
ultralytics.checks()

Ultralytics 8.3.102 🚀 Python-3.11.11 torch-2.6.0+cu124 CUDA:0 (NVIDIA A100-SXM4-40GB, 40507MiB)
Setup complete ✅ (12 CPUs, 83.5 GB RAM, 41.2/235.7 GB disk)


In [ ]:
import os
HOME = os.getcwd()
print(HOME)

/content


In [ ]:
WEIGHTS_PATH = f"{HOME}/weights"
!mkdir -p {WEIGHTS_PATH}
!wget -P {WEIGHTS_PATH} -q https://github.com/ultralytics/assets/releases/download/v8.3.0/yolo11n.pt

In [ ]:
from google.colab import drive
import zipfile
import os

# Mount Google Drive
drive.mount('/content/drive')

# Define HOME directory
HOME = '/content'

# Create a dedicated directory for datasets
!mkdir -p {HOME}/datasets

# Change working directory to datasets folder
%cd {HOME}/datasets

# Path to your dataset zip file in Google Drive
zip_path = "/content/drive/My Drive/Pistachios/New-Pistachios.v3i.yolov8.zip"

# Unzipping the dataset to the datasets directory
with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall(f'{HOME}/datasets')

# List the contents of the datasets directory to verify extraction
!ls {HOME}/datasets

# Now directly using the paths to train, valid, and test inside datasets
!ls {HOME}/datasets/train || echo "Folder does not exist. Listing contents of /content/datasets to check the structure:"


Mounted at /content/drive
/content/datasets
data.yaml  README.roboflow.txt	test  train  valid
images	labels


In [ ]:
import os

# Define HOME directory (assumed to be '/content')
HOME = '/content'

splits = ['train', 'valid', 'test']

# Overall totals
overall_bg_annotations = 0
overall_pistachio_annotations = 0
overall_total_annotations = 0
overall_total_images = 0

for split in splits:
    # Directories for each split
    annotations_dir = os.path.join(HOME, 'datasets', split, 'labels')
    images_dir = os.path.join(HOME, 'datasets', split, 'images')

    # Counters for this split
    bg_count = 0          # Count for background (class_id=0)
    pistachio_count = 0   # Count for pistachio (class_id=1)
    total_annotations = 0
    image_count = 0

    # Count annotations in each label file
    if os.path.exists(annotations_dir):
        for file_name in os.listdir(annotations_dir):
            if file_name.endswith('.txt'):
                file_path = os.path.join(annotations_dir, file_name)
                with open(file_path, 'r') as file:
                    lines = file.readlines()
                    for line in lines:
                        line = line.strip()
                        if not line:
                            continue
                        parts = line.split()
                        class_id = int(parts[0])
                        total_annotations += 1
                        if class_id == 0:
                            bg_count += 1
                        elif class_id == 1:
                            pistachio_count += 1

    # Count images in the images directory
    if os.path.exists(images_dir):
        for file_name in os.listdir(images_dir):
            if file_name.lower().endswith(('.jpg', '.jpeg', '.png', '.bmp', '.tiff')):
                image_count += 1

    # Update overall totals
    overall_bg_annotations += bg_count
    overall_pistachio_annotations += pistachio_count
    overall_total_annotations += total_annotations
    overall_total_images += image_count

    # Print split summary
    print(f"{split.upper()} set:")
    print(f"  Images: {image_count}")
    print(f"  Total Annotations: {total_annotations}")
    print(f"    - Background (class_id=0): {bg_count}")
    print(f"    - Pistachios (class_id=1): {pistachio_count}")
    print("")

# Print overall summary
print("==== OVERALL TOTALS ====")
print(f"Total Images: {overall_total_images}")
print(f"Total Annotations: {overall_total_annotations}")
print(f"  - Background (class_id=0): {overall_bg_annotations}")
print(f"  - Pistachios (class_id=1): {overall_pistachio_annotations}")

TRAIN set:
  Images: 500
  Total Annotations: 18765
    - Background (class_id=0): 500
    - Pistachios (class_id=1): 18265

VALID set:
  Images: 30
  Total Annotations: 1220
    - Background (class_id=0): 30
    - Pistachios (class_id=1): 1190

TEST set:
  Images: 15
  Total Annotations: 571
    - Background (class_id=0): 15
    - Pistachios (class_id=1): 556

==== OVERALL TOTALS ====
Total Images: 545
Total Annotations: 20556
  - Background (class_id=0): 545
  - Pistachios (class_id=1): 20011


In [ ]:
import os
import shutil
import random
import uuid
from PIL import Image

# ===================== CONFIGURATION ======================
HOME = '/content'

# Paths to the original dataset splits (each with "images/" and "labels/" subfolders)
original_dataset_dir = os.path.join(HOME, 'datasets')
splits = ['train', 'valid', 'test']

# Directory to unify all images and labels before processing
unified_dir = os.path.join(HOME, 'unified_dataset')
unified_img_dir = os.path.join(unified_dir, 'images')
unified_lbl_dir = os.path.join(unified_dir, 'labels')
os.makedirs(unified_img_dir, exist_ok=True)
os.makedirs(unified_lbl_dir, exist_ok=True)

# YOLO class IDs in your dataset (before remapping)
BACKGROUND_CLASS_ID = 0     # original background
PISTACHIO_CLASS_ID  = 1     # original pistachio

# Final single-class YOLO config: after background removal, only one class (pistachios) remains.
train_ratio = 0.8
val_ratio   = 0.1
test_ratio  = 0.1

# Directories for final train/valid/test splits
base_dir = os.path.join(HOME, 'working_datasets')
train_img_dir = os.path.join(base_dir, 'train', 'images')
train_lbl_dir = os.path.join(base_dir, 'train', 'labels')
val_img_dir   = os.path.join(base_dir, 'valid', 'images')
val_lbl_dir   = os.path.join(base_dir, 'valid', 'labels')
test_img_dir  = os.path.join(base_dir, 'test', 'images')
test_lbl_dir  = os.path.join(base_dir, 'test', 'labels')

for d in [train_img_dir, train_lbl_dir, val_img_dir, val_lbl_dir, test_img_dir, test_lbl_dir]:
    os.makedirs(d, exist_ok=True)

# ===================== STEP 1: UNIFY ALL IMAGES =====================
print("Combining all splits (train/valid/test) into one 'unified_dataset' folder...")

for split in splits:
    split_img_dir = os.path.join(original_dataset_dir, split, 'images')
    split_lbl_dir = os.path.join(original_dataset_dir, split, 'labels')
    if not os.path.isdir(split_img_dir):
        print(f"Warning: {split_img_dir} not found; skipping split '{split}'")
        continue

    for img_file in os.listdir(split_img_dir):
        if not img_file.lower().endswith(('.jpg', '.jpeg', '.png')):
            continue
        src_img_path = os.path.join(split_img_dir, img_file)
        # Use the full base name (do not strip off anything)
        base_name = os.path.splitext(img_file)[0]
        lbl_file = base_name + '.txt'
        src_lbl_path = os.path.join(split_lbl_dir, lbl_file) if os.path.isfile(os.path.join(split_lbl_dir, lbl_file)) else None

        # Copy image into unified folder
        dst_img_path = os.path.join(unified_img_dir, img_file)
        shutil.copy2(src_img_path, dst_img_path)

        # Copy corresponding label file if it exists
        if src_lbl_path and os.path.isfile(src_lbl_path):
            dst_lbl_path = os.path.join(unified_lbl_dir, lbl_file)
            shutil.copy2(src_lbl_path, dst_lbl_path)

print("All splits have been merged into:", unified_dir)
print("")

# ===================== STEP 2: BACKGROUND REMOVAL =====================
def read_yolo_annotation(file_path):
    """Reads YOLO-format annotations (bounding boxes or polygons)."""
    annotations = []
    if not os.path.exists(file_path):
        return annotations
    with open(file_path, 'r') as f:
        for line in f:
            line = line.strip()
            if not line:
                continue
            parts = line.split()
            class_id = int(parts[0])
            coords = list(map(float, parts[1:]))
            if len(coords) == 4:
                x_c, y_c, w, h = coords
                annotations.append((class_id, x_c, y_c, w, h))
            else:
                if len(coords) < 6 or len(coords) % 2 != 0:
                    print(f"Skipping malformed polygon line in {file_path}: '{line}'")
                    continue
                xs = coords[0::2]
                ys = coords[1::2]
                x_min, x_max = min(xs), max(xs)
                y_min, y_max = min(ys), max(ys)
                x_c = (x_min + x_max) / 2.0
                y_c = (y_min + y_max) / 2.0
                w   = x_max - x_min
                h   = y_max - y_min
                annotations.append((class_id, x_c, y_c, w, h))
    return annotations

def save_yolo_annotation(file_path, annotations):
    """Writes YOLO-format annotations to a file."""
    with open(file_path, 'w') as f:
        for ann in annotations:
            class_id, x_c, y_c, w, h = ann
            f.write(f"{class_id} {x_c:.6f} {y_c:.6f} {w:.6f} {h:.6f}\n")

all_images = [img for img in os.listdir(unified_img_dir) if img.lower().endswith(('.jpg', '.jpeg', '.png'))]

num_bg_crops_created = 0
num_images_with_pistachio = 0
num_negative_images = 0

print("Removing 'background' bounding boxes (class_id=0) and cropping them into new negative images...")

for img_name in all_images:
    label_path = os.path.join(unified_lbl_dir, os.path.splitext(img_name)[0] + '.txt')
    if not os.path.exists(label_path):
        num_negative_images += 1
        continue
    annotations = read_yolo_annotation(label_path)
    if not annotations:
        num_negative_images += 1
        continue
    image_path = os.path.join(unified_img_dir, img_name)
    img = Image.open(image_path).convert('RGB')
    img_w, img_h = img.size
    updated_annotations = []
    for i, ann in enumerate(annotations):
        class_id, x_c, y_c, w, h = ann
        if class_id == BACKGROUND_CLASS_ID:
            x_center = x_c * img_w
            y_center = y_c * img_h
            bbox_w   = w * img_w
            bbox_h   = h * img_h
            x_min = int(x_center - bbox_w / 2)
            x_max = int(x_center + bbox_w / 2)
            y_min = int(y_center - bbox_h / 2)
            y_max = int(y_center + bbox_h / 2)
            cropped = img.crop((x_min, y_min, x_max, y_max))
            base_name = os.path.splitext(img_name)[0]
            unique_id = uuid.uuid4().hex
            crop_img_name = f"{base_name}_bg_{i}_{unique_id}.jpg"
            crop_img_path = os.path.join(unified_img_dir, crop_img_name)
            cropped.save(crop_img_path)
            crop_label_path = os.path.join(unified_lbl_dir, os.path.splitext(crop_img_name)[0] + '.txt')
            open(crop_label_path, 'w').close()
            num_bg_crops_created += 1
        else:
            new_class_id = 0
            updated_annotations.append((new_class_id, x_c, y_c, w, h))
    save_yolo_annotation(label_path, updated_annotations)
    if len(updated_annotations) == 0:
        num_negative_images += 1
    else:
        num_images_with_pistachio += 1

print("Background removal completed!")
print(f"  Number of new negative crop images: {num_bg_crops_created}")
print(f"  Images with at least one pistachio: {num_images_with_pistachio}")
print(f"  Pure negative images (no bboxes):   {num_negative_images}")
print("")

# ===================== STEP 3: STRATIFIED SPLIT =====================
all_images = [img for img in os.listdir(unified_img_dir) if img.lower().endswith(('.jpg', '.jpeg', '.png'))]

positive_images = []
negative_images = []

def has_annotations(label_path):
    if not os.path.isfile(label_path):
        return False
    ann = read_yolo_annotation(label_path)
    return (len(ann) > 0)

for img_name in all_images:
    label_name = os.path.splitext(img_name)[0] + '.txt'
    label_path = os.path.join(unified_lbl_dir, label_name)
    if has_annotations(label_path):
        positive_images.append(img_name)
    else:
        negative_images.append(img_name)

def split_counts(total):
    t = int(total * train_ratio)
    v = int(total * val_ratio)
    s = total - t - v
    return t, v, s

random.shuffle(positive_images)
random.shuffle(negative_images)

pos_train_count, pos_val_count, pos_test_count = split_counts(len(positive_images))
neg_train_count, neg_val_count, neg_test_count = split_counts(len(negative_images))

pos_train = positive_images[:pos_train_count]
pos_val   = positive_images[pos_train_count: pos_train_count + pos_val_count]
pos_test  = positive_images[pos_train_count + pos_val_count:]
neg_train = negative_images[:neg_train_count]
neg_val   = negative_images[neg_train_count: neg_train_count + neg_val_count]
neg_test  = negative_images[neg_train_count + neg_val_count:]

train_images = pos_train + neg_train
val_images   = pos_val + neg_val
test_images  = pos_test + neg_test

random.shuffle(train_images)
random.shuffle(val_images)
random.shuffle(test_images)

print("=== Stratified Split Results ===")
print(f"  Train: {len(train_images)} total = {len(pos_train)} positive + {len(neg_train)} negative")
print(f"  Valid: {len(val_images)} total  = {len(pos_val)} positive + {len(neg_val)} negative")
print(f"  Test:  {len(test_images)} total = {len(pos_test)} positive + {len(neg_test)} negative")
print("")

# ===================== STEP 4: MOVE FINAL SPLITS =====================
def move_files(image_list, img_dest_dir, lbl_dest_dir):
    for img_file in image_list:
        src_img_path = os.path.join(unified_img_dir, img_file)
        if not os.path.exists(src_img_path):
            print(f"Warning: {src_img_path} not found, skipping.")
            continue
        base_name = os.path.splitext(img_file)[0]  # Use the full base name (unchanged)
        lbl_file = base_name + '.txt'
        src_lbl_path = os.path.join(unified_lbl_dir, lbl_file)
        dst_img_path = os.path.join(img_dest_dir, img_file)
        dst_lbl_path = os.path.join(lbl_dest_dir, lbl_file)
        if os.path.exists(dst_img_path):
            os.remove(dst_img_path)
        shutil.move(src_img_path, dst_img_path)
        if os.path.exists(dst_lbl_path):
            os.remove(dst_lbl_path)
        if os.path.exists(src_lbl_path):
            shutil.move(src_lbl_path, dst_lbl_path)
        else:
            with open(dst_lbl_path, 'w') as f:
                f.write('')

print("Moving files to final train/valid/test directories...")
move_files(train_images, train_img_dir, train_lbl_dir)
move_files(val_images,   val_img_dir,   val_lbl_dir)
move_files(test_images,  test_img_dir,  test_lbl_dir)

print("Final dataset splits created in:", base_dir)
print("")
print("===== SUMMARY =====")
print(f"Train: {len(os.listdir(train_img_dir))} images -> {train_img_dir}")
print(f"Valid: {len(os.listdir(val_img_dir))} images -> {val_img_dir}")
print(f"Test:  {len(os.listdir(test_img_dir))} images -> {test_img_dir}")
print("")
print("Data config (data.yaml) example:")
print("nc: 1")
print("names: ['pistachios']")
print("train: ../train/images")
print("val:   ../valid/images")
print("test:  ../test/images")
print("")
print("Done! You now have a single-class dataset with no background bounding boxes,")
print("cropped negative samples, and an 80/10/10 stratified split stored in /content/working_datasets.")

Combining all splits (train/valid/test) into one 'unified_dataset' folder...
All splits have been merged into: /content/unified_dataset

Removing 'background' bounding boxes (class_id=0) and cropping them into new negative images...
Background removal completed!
  Number of new negative crop images: 545
  Images with at least one pistachio: 545
  Pure negative images (no bboxes):   0

=== Stratified Split Results ===
  Train: 872 total = 436 positive + 436 negative
  Valid: 108 total  = 54 positive + 54 negative
  Test:  110 total = 55 positive + 55 negative

Moving files to final train/valid/test directories...
Final dataset splits created in: /content/working_datasets

===== SUMMARY =====
Train: 872 images -> /content/working_datasets/train/images
Valid: 108 images -> /content/working_datasets/valid/images
Test:  110 images -> /content/working_datasets/test/images

Data config (data.yaml) example:
nc: 1
names: ['pistachios']
train: ../train/images
val:   ../valid/images
test:  ../tes

In [ ]:
import os

# Path to your datasets directory and desired data.yaml file
base_dir = '/content/working_datasets'
data_yaml_path = os.path.join(base_dir, 'data.yaml')

# Write out the updated data.yaml
with open(data_yaml_path, 'w+') as f:
    f.write("train: ../train/images\n")
    f.write("val: ../valid/images\n")
    f.write("test: ../test/images\n\n")
    f.write("nc: 1\n")
    f.write("names: ['Pistachios']\n\n")

    # If you still want the Roboflow metadata, include it, otherwise remove this block:
    f.write("roboflow:\n")
    f.write("  workspace: pistachios-knen1\n")
    f.write("  project: new-pistachios-zoodo\n")
    f.write("  version: 2\n")
    f.write("  license: Private\n")
    f.write("  url: https://app.roboflow.com/pistachios-knen1/new-pistachios-zoodo/2\n")

print("data.yaml updated with single-class configuration.")


data.yaml updated with single-class configuration.


In [ ]:
from ultralytics import YOLO
import os
import pandas as pd

# Define the hyperparameter grid for grid search
param_grid = {
    "epochs": [25, 50, 100],  # Number of training epochs
    "imgsz": [640, 800],       # Image sizes
    "batch": [16, 32],         # Batch sizes
}

# Path to the YOLO model weights and dataset config
model_path = f"{WEIGHTS_PATH}/yolo11n.pt"
data_path = f"{HOME}/working_datasets/data.yaml"

# Initialize results list to store metrics for each run
results_summary = []
train_num = 1  # Initialize train number counter

# Perform grid search over hyperparameters
for epochs in param_grid["epochs"]:
    for imgsz in param_grid["imgsz"]:
        for batch in param_grid["batch"]:
            print(f"\nTraining run {train_num}: epochs={epochs}, imgsz={imgsz}, batch={batch}\n")

            # Initialize the YOLO model
            model = YOLO(model_path)

            # Train the model with current hyperparameters
            train_results = model.train(
                data=data_path,
                epochs=epochs,
                imgsz=imgsz,
                batch=batch,
                device="cuda",
                plots=False  # Set to True if you want plots for each run
            )

            # Validate the model to get mAP@50
            val_results = model.val()  # Validate using the dataset specified in `data.yaml`
            map50 = val_results.box.map50  # mAP@50 from validation

            # Save the configuration, train number, and results
            results_summary.append({
                "train_num": train_num,
                "epochs": epochs,
                "imgsz": imgsz,
                "batch": batch,
                "mAP@50": map50,
            })

            print(f"Finished Training run {train_num}: epochs={epochs}, imgsz={imgsz}, batch={batch}, mAP@50={map50}")

            train_num += 1  # Increment train number for next run

# Find the best configuration based on mAP@50
best_config = max(results_summary, key=lambda x: x["mAP@50"])

print("\nBest Configuration Found:")
print(best_config)

# Save results to a CSV file (optional)
results_df = pd.DataFrame(results_summary)
results_df.to_csv("grid_search_results_negative_samples.csv", index=False)

print("\nGrid search results saved to 'grid_search_results_negative_samples.csv'")


Training run 1: epochs=25, imgsz=640, batch=16

Ultralytics 8.3.102 🚀 Python-3.11.11 torch-2.6.0+cu124 CUDA:0 (NVIDIA A100-SXM4-40GB, 40507MiB)
engine/trainer: task=detect, mode=train, model=/content/weights/yolo11n.pt, data=/content/working_datasets/data.yaml, epochs=25, time=None, patience=100, batch=16, imgsz=640, save=True, save_period=-1, cache=False, device=cuda, workers=8, project=None, name=train, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=False, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False, save

100%|██████████| 755k/755k [00:00<00:00, 94.5MB/s]


Overriding model.yaml nc=80 with nc=1

                   from  n    params  module                                       arguments                     
  0                  -1  1       464  ultralytics.nn.modules.conv.Conv             [3, 16, 3, 2]                 
  1                  -1  1      4672  ultralytics.nn.modules.conv.Conv             [16, 32, 3, 2]                
  2                  -1  1      6640  ultralytics.nn.modules.block.C3k2            [32, 64, 1, False, 0.25]      
  3                  -1  1     36992  ultralytics.nn.modules.conv.Conv             [64, 64, 3, 2]                
  4                  -1  1     26080  ultralytics.nn.modules.block.C3k2            [64, 128, 1, False, 0.25]     
  5                  -1  1    147712  ultralytics.nn.modules.conv.Conv             [128, 128, 3, 2]              
  6                  -1  1     87040  ultralytics.nn.modules.block.C3k2            [128, 128, 1, True]           
  7                  -1  1    295424  ultralytics

100%|██████████| 5.35M/5.35M [00:00<00:00, 326MB/s]


AMP: checks passed ✅


...

                   all        108       1863       0.87      0.875      0.944      0.634
Speed: 0.2ms preprocess, 0.7ms inference, 0.0ms loss, 0.7ms postprocess per image


Ultralytics 8.3.102 🚀 Python-3.11.11 torch-2.6.0+cu124 CUDA:0 (NVIDIA A100-SXM4-40GB, 40507MiB)
YOLO11n summary (fused): 100 layers, 2,582,347 parameters, 0 gradients, 6.3 GFLOPs


val: Scanning /content/working_datasets/valid/labels.cache... 108 images, 54 backgrounds, 0 corrupt: 100%|██████████| 108/108 [00:00<?, ?it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 4/4 [00:01<00:00,  2.95it/s]

                   all        108       1863      0.871      0.869      0.945       0.64
Speed: 0.2ms preprocess, 1.0ms inference, 0.0ms loss, 1.0ms postprocess per image


Finished Training run 12: epochs=100, imgsz=800, batch=32, mAP@50=0.9447397153975561

Best Configuration Found:
{'train_num': 12, 'epochs': 100, 'imgsz': 800, 'batch': 32, 'mAP@50': np.float64(0.9447397153975561)}

Grid search results saved to 'grid_search_results_negative_samples.csv'
